# Author: Blai Ras
# Vector Autoregressive with exogenous variables (VARX)

### From the master thesis 

### <center>  _'Accuracy comparison between Sparse Autoregressive and XGBoost models for high-dimensional product sales forecasting'_ </center>

In this notebook we present the building, training and evaluation of all the segments using the VAR model. All the needed documentation and detailed description can be found in the thesis paper.

## Needed libraries

In [ ]:
library(bigtime)
library(Metrics)

## Support functions

This first function receives a dataframe and returns the index of the first exogenous variable. Therefore, since the endogenous sales variables are ordered, we can perform the splitting between sales and promotion variables. 

In [2]:
find.endogenous.index <- function(data) {
    # Finding the number of endogenous variables (only the ones who are sales)
    logical = grepl("Sales" , names(data))
    num.endogenous <- length(logical[logical == TRUE])
    num.endogenous <- num.endogenous+1
    return(num.endogenous)
}

The scaler function receives a dataframe and returns 3 arguments: the same dataframe standardized (zero mean and unit standard deviation) and the original standard deviation and mean, so we can recover the standardization.

In [3]:
scaler <- function(data) {
    scaled <- scale(data)
    return(list("data"=scaled,
               "std"=attr(scaled,'scaled:scale'),
               "mean"=attr(scaled, 'scaled:center')))
}

This function performs the model building step. It receives the endogenous variable matrix (Y), the exogenous variable matrix (X) and a horizon forecast (h=4). Then, iteratively, builds a model with the specified configuration for each forecast horizon, saving them in an array which we return.

In [4]:
model.building <- function(Y,X,h){
    models <-list()
    for (i in 1:h) {
        cat(paste0("\nIteration ",i))
        flush.console()
        VARXfit_cv <- sparseVARX(Y=Y,
                                 X=X,
                                 selection = "cv",
                                 h=i,
                                 check_std = FALSE)
        models[[i]] <- VARXfit_cv
    }
    return(models)
}

This function performs the prediction step. It receives the list of models and a forecast horizon, h. Then, for each forecast horizon we call its model pair, and save the predictors in a matrix, which we return.

In [5]:
predict.all <- function(models,h) {
    preds <- directforecast(models[[1]])
    for (i in 2:h) {
        preds <- cbind(preds,directforecast(models[[i]],h=i))
    }
    return(as.matrix(t(preds)))
}

This two functions are modified evaluation methods. As we can see, the modification is done in both of their denominators, so when an actual value (y_true) is exactly zero, we don't return a NaN. 

In [6]:
MAAPE <- function(y_true, y_pred) {
    return(mean(atan(abs((y_true-y_pred)/(y_true+0.000000000000001))))*100)
}

MAPE.divide.zero <- function(y_true, y_pred) {
  MAPE <- mean(abs((y_true - y_pred) / (y_true+0.000000000000001)))*100
  return(MAPE)
}

This following function performs the evaluation part. It receives the segment identifier (an integer), the prediction matrix, the test/actual values, the time needed for the model training and a boolean indicating if we need to evaluate standardized or not predictions.

In [7]:
evaluate <- function(segment,preds,test,time,is.scaled) {
    
    #These are the monthly forecast and actual values.
    tSum <- colSums(test)
    pSum <- colSums(preds)
    
    #The results vector with the variables that we will output on the .csv file: the segment identifier,  weekly RMSE, 
    #MAAPE, MASE, MAPE and MAE, and then repeated again for the monthly errors. Finally, the time and the number of 
    #variables of the segment.
    results <- c(segment,
                 round(rmse(test,preds),2),round(MAAPE(test,preds),2),round(mase(test,preds),2),
                 round(MAPE.divide.zero(test,preds),2),round(mae(test,preds),2),
                 round(rmse(tSum,pSum),2),round(MAAPE(tSum,pSum),2),round(mase(tSum,pSum),2),
                 round(MAPE.divide.zero(tSum,pSum),2),round(mae(tSum,pSum),2),
                 ncol(preds),
                 time)
    
    #If we performed the evaluation with the predictions standardized, we save it to a file named just 'evaluation'.
    #If not, the file is 'Raw Evaluation'
    if (is.scaled) {
        write.table(t(results), file = 'evaluation.csv', append = TRUE, row.names = FALSE, 
                    col.names = FALSE, sep = ",")
    } else {
        write.table(t(results), file = 'Raw Evaluation.csv', append = TRUE, row.names = FALSE, 
                    col.names = FALSE, sep = ",")
    }
    
}

This function controls the overall process of data preprocessing, model building & training and evaluation.

First, we list all the training files with extension .csv in the indicated folder. Due that all these files have their SKU id on their name, we extract it using regex.

For each SKU we have train and test files. That's why we iterate 2 by 2. So, for each SKU, we read their test pre-processed file, their original test file and their train file, unless the model already exists, which we check by looking at the Models folder.

If not, then we get its number of endogenous and exogenous variables using the function ``find.endogenous.index``. Thanks to this index, we perform the split of endogenous and exogenous variables. We scale each matrices with our function ``scaler``. With the generated standard deviation and mean, we also scale the test values.

We continue by creating and training the model, using the function ``model.building``. After the training step is done, we save the model using the .RDS file extension. 

Before proceeding with the evalatuion, we undo the first-differentiation to test our model re-escalating the predictions. In order to do so, we recover the last sales of our train set and use them with the function ``diffinv``. Now we can evaluate with standardization and without.

In [8]:
train.all <- function() {
    
    #Load all files
    file.list.processed <- list.files(path="Data/Processed Train & Test Sets",pattern="*.csv",full.names=TRUE)

    #Write the header of the evaluation results files
    Metrics <- c("Segment","Weekly RMSE","Weekly MAAPE","Weekly MASE","Weekly MAPE","Weekly MAE",
                "Monthly RMSE", "Monthly MAAPE", "Monthly MASE", "Monthly MAPE", "Monthly MAE", "Variables","Time")
    write.table(t(Metrics), file = 'evaluation.csv', append = TRUE, row.names = FALSE, col.names = FALSE, sep = ",")
    write.table(t(Metrics), file = 'Raw Evaluation.csv', append = TRUE, row.names = FALSE, col.names = FALSE, sep = ",")
    
    for(i in seq(from=1, to=length(file.list.processed), by=2)) {
        
        segment = as.numeric(gsub("[^\\d]+", "", file.list.processed[[i]], perl=TRUE))
        if (file.exists(paste0("Models/Processed/",segment,".rds"))) {
            print(paste0("Segment ",segment," already trained, skipping."))
            next
        }
        cat(paste0('\nStart of Segment:',segment),'\n')
        
        #Read the data
        test <- read.csv(paste0("Data/Processed Train & Test Sets/",segment,"Test.csv"))
        train <- read.csv(paste0("Data/Processed Train & Test Sets/",segment,"Train.csv"))
        test.raw <- read.csv(paste0("Data/Raw Train & Test Sets/",segment,"Test.csv"))
        
        print(paste0("Dimensions:",dim(train)))
        
        #Get the number of variables of the problem
        endogenous.index <- find.endogenous.index(train)
              
        print(paste0("Num. of endogenous variables:",endogenous.index))
        
        #Split the train sets by endogenous and exogenous variables
        train.endo <- train[,2:endogenous.index]
        train.exo <- train[,(endogenous.index+1):ncol(train)]
        
        #Data Scaling
        #Train sets
        scaled.endo <- scaler(train.endo)
        scaled.exo <- scaler(train.exo)
        
        #Just in case there's a column with all of its values to zero (no discount, no duration)
        scaled.exo$data[is.nan(scaled.exo$data)] = 0
        
        #Test set
        scaled.test <- (test[,2:endogenous.index] - scaled.endo$mean) / scaled.endo$std
        
        ptm <- proc.time()
        
        #Build the model
        models <- model.building(Y=scaled.endo$data,
                                 X=scaled.exo$data,
                                 h=4)
        
        time <- proc.time() - ptm
        print(time)
        
        #Save the model
        saveRDS(models, paste0("Models/Processed/",segment,".rds"))
        
        #Forecast without scaling
        preds <- predict.all(models,h=4)
        
        #Evaluate
        evaluate(segment,preds,as.matrix(scaled.test),time[[3]],is.scaled=TRUE)
        
        # Undo Scaling
        unscaled.preds <- (preds * scaled.endo$std) + scaled.endo$mean

        # Undo First Differenciation
        last.sales <- read.csv(paste0("Data/Raw Train & Test Sets/",segment,"Train.csv"))[nrow(train)+1,2:endogenous.index]
        original.preds <- diffinv(unscaled.preds,xi=last.sales)[-1,]

        #Evaluate without scaling
        evaluate(segment,original.preds,as.matrix(test.raw[,2:endogenous.index]),time[[3]],is.scaled=FALSE)
        
    }
}

In [ ]:
train.all()

## Model evaluation standalone, without training

This next functions perform only the model evaluation, once each model is built. They were built for test purposes, since we don't need to re-train again all the models.

This first function is pretty similar to the previous evaluation one. It has been adapted to save the results in another temporal files and to perform the evaluation with or without rounding, for test purposes.

In [9]:
evaluate.one <- function(segment,preds,test,is.scaled,rounding=FALSE) {
    
    if (rounding) {
        test <- round(test)
        preds <- round(preds)
    }
    
    tSum <- colSums(test)
    pSum <- colSums(preds)
    #Metrics <- c("Segment","Weekly RMSE","Weekly MAAPE","Weekly MASE","Weekly MAPE","Weekly MAE",
    #             "Monthly RMSE", "Monthly MAAPE", "Monthly MASE", "Monthly MAPE", "Monthly MAE", "Variables","Time")
    results <- c(segment,
                 round(rmse(test,preds),2),round(MAAPE(test,preds),2),round(mase(test,preds),2),
                 round(MAPE.divide.zero(test,preds),2),round(mae(test,preds),2),
                 
                 round(rmse(tSum,pSum),2),round(MAAPE(tSum,pSum),2),round(mase(tSum,pSum),2),
                 round(MAPE.divide.zero(tSum,pSum),2),round(mae(tSum,pSum),2),
                 ncol(preds))

    if (is.scaled) {
        write.table(t(results), file = 'temp.csv', append = TRUE, row.names = FALSE, 
                    col.names = FALSE, sep = ",")
    } else {
        write.table(t(results), file = 'Raw temp.csv', append = TRUE, row.names = FALSE, 
                    col.names = FALSE, sep = ",")
    }
    
}

And this other function is quite similar to the ``train.all()``, but without the model building & training part. Therefore, it just reads the training, test and model files for performing the evaluation.

In [10]:
evaluate.models <- function(rounding) {
    
    #Load all files
    file.list.processed <- list.files(path="Data/Processed Train & Test Sets",pattern="*.csv",full.names=TRUE)

    #Write the header of the evaluation results files
    Metrics <- c("Segment","Weekly RMSE","Weekly MAAPE","Weekly MASE","Weekly MAPE","Weekly MAE",
                 "Monthly RMSE", "Monthly MAAPE", "Monthly MASE", "Monthly MAPE", "Monthly MAE", "Variables","Time")
    write.table(t(Metrics), file = 'temp.csv', append = TRUE, row.names = FALSE, col.names = FALSE, sep = ",")
    write.table(t(Metrics), file = 'Raw temp.csv', append = TRUE, row.names = FALSE, col.names = FALSE, sep = ",")
    
    for(i in seq(from=1, to=length(file.list.processed), by=2)) {
        
        segment = as.numeric(gsub("[^\\d]+", "", file.list.processed[[i]], perl=TRUE))
        
        cat(paste0('\nStart of Segment:',segment),'\n')
        
        #Read the data
        test <- read.csv(paste0("Data/Processed Train & Test Sets/",segment,"Test.csv"))
        train <- read.csv(paste0("Data/Processed Train & Test Sets/",segment,"Train.csv"))
        test.raw <- read.csv(paste0("Data/Raw Train & Test Sets/",segment,"Test.csv"))
        
        print(paste0("Dimensions:",dim(train)))
        
        #Get the number of variables of the problem
        endogenous.index <- find.endogenous.index(train)
              
        print(paste0("Num. of endogenous variables:",endogenous.index))
        
        #Split the train sets by endogenous and exogenous variables
        train.endo <- train[,2:endogenous.index]
        train.exo <- train[,(endogenous.index+1):ncol(train)]
        
        #Data Scaling
        #Train sets
        scaled.endo <- scaler(train.endo)
        scaled.exo <- scaler(train.exo)
        
        #Just in case there's a column with all of its values to zero (no discount, no duration)
        scaled.exo$data[is.nan(scaled.exo$data)] = 0
        
        #Test set
        scaled.test <- (test[,2:endogenous.index] - scaled.endo$mean) / scaled.endo$std
        
        #Model recover
        models <- readRDS(paste0("Models/Processed/",segment,".RDS"))
        
        #Forecast without scaling
        preds <- predict.all(models,h=4)
        
        #Evaluate
        evaluate.one(segment,preds,as.matrix(scaled.test),is.scaled=TRUE,rounding=rounding)
        
        # Undo Scaling
        unscaled.preds <- (preds * scaled.endo$std) + scaled.endo$mean

        # Undo First Differenciation
        last.sales <- read.csv(paste0("Data/Raw Train & Test Sets/",segment,"Train.csv"))[nrow(train)+1,2:endogenous.index]
        original.preds <- diffinv(unscaled.preds,xi=last.sales)[-1,]

        #Evaluate without scaling
        evaluate.one(segment,original.preds,as.matrix(test.raw[,2:endogenous.index]),is.scaled=FALSE,rounding=rounding)
        
    }
    
    
}

In [11]:
evaluate.models(rounding=FALSE)


Start of Segment:10120 
[1] "Dimensions:150"  "Dimensions:1204"
[1] "Num. of endogenous variables:402"

Start of Segment:10140 
[1] "Dimensions:150" "Dimensions:49" 
[1] "Num. of endogenous variables:17"

Start of Segment:10150 
[1] "Dimensions:150"  "Dimensions:1534"
[1] "Num. of endogenous variables:512"

Start of Segment:10160 
[1] "Dimensions:150" "Dimensions:133"
[1] "Num. of endogenous variables:45"

Start of Segment:10170 
[1] "Dimensions:150" "Dimensions:952"
[1] "Num. of endogenous variables:318"

Start of Segment:10180 
[1] "Dimensions:150" "Dimensions:343"
[1] "Num. of endogenous variables:115"

Start of Segment:11902 
[1] "Dimensions:150" "Dimensions:46" 
[1] "Num. of endogenous variables:16"

Start of Segment:11903 
[1] "Dimensions:150" "Dimensions:523"
[1] "Num. of endogenous variables:175"

Start of Segment:11908 
[1] "Dimensions:150"  "Dimensions:1078"
[1] "Num. of endogenous variables:360"

Start of Segment:11920 
[1] "Dimensions:150" "Dimensions:466"
[1] "Num. of end